<a href="https://colab.research.google.com/github/cuongvng/neural-networks-with-PyTorch/blob/master/RNNs/PlainRNN/Plain_RNN.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [9]:
!pip install git+https://github.com/cuongvng/neural-networks-with-PyTorch

  Cloning https://github.com/cuongvng/neural-networks-with-PyTorch to /private/var/folders/gl/y27tnps503v1xfnl2xpl5ybc0000gn/T/pip-req-build-rzbwc6j6
  Running command git clone -q https://github.com/cuongvng/neural-networks-with-PyTorch /private/var/folders/gl/y27tnps503v1xfnl2xpl5ybc0000gn/T/pip-req-build-rzbwc6j6
  Created wheel for train-plot-utils: filename=train_plot_utils-1.0-py3-none-any.whl size=5457 sha256=0dc79d15de909f91594c26cd82d64702b81253820431d0491e636171bc173b53
  Stored in directory: /private/var/folders/gl/y27tnps503v1xfnl2xpl5ybc0000gn/T/pip-ephem-wheel-cache-vzzppyy4/wheels/f7/47/17/cc4fb90864ec83d108689f208b7f6005c159e50f873674997d
Successfully built train-plot-utils


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import sys
sys.path.append("../..")
from utils.rnn_dataset_creator import ForexDataset
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'utils.rnn_dataset_creator'

Load time series data